# New Section

In [ ]:
!pip install nltk
import nltk
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
!pip install vaderSentiment
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import csv
import nltk


nltk.download('stopwords')
nltk.download('punkt')

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

custom_words = dict ()


def get_dictionary():                                                 ### UPDATING THE VADER BY OUR CUSTOM MADE DICTIPNARY SPECIALLY FOR TESLA TWEETS!!
    file_path = '/content/neg_1000_78.28_SCALED.csv'
    df = pd.read_csv(file_path)
    for i in range(len(df)):
        custom_words[df['Word'][i]] = df['robust_scaled'][i]


get_dictionary()
analyzer.lexicon.update(custom_words)
print(custom_words)
print("dictionary updated")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 1.0 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


{'lol': 16557.1779632912, 'looks': 2185.57156487876, 'like': 35937.039064194694, 'rush': 22.372421083430385, 'buy': 59302.66581385829, 'resistance': 1198.7224748855576, 'maybe': 853.1859412323288, 'getting': 2603.472569341356, 'ready': 2333.8087052264964, 'wipe': 3.1388915487846885, 'shirts': 4.520954791473693, 'still': 8244.117464742569, 'bargain': 7.042091251620173, 'price': 7263.991537204086, 'going': 37123.721863481456, 'robinhood': 97.4782068568694, 'market': 24248.87176408726, 'order': 414.9341007641556, 'wont': 595.0182468942461, 'even': 3620.599087057408, 'go': 49449.506544068994, 'thru': 36.32751654770194, 'minutes': 172.9109888852544, 'pending': 12.98446108991123, 'gme': 71693.27192756727, 'pushing': 98.09105956265536, 'come': 9569.745448246556, 'baby': 1960.2953627407412, 'keep': 3917.5991805154463, 'recovering': 12.392966052057831, 'pleeease': 0.0515815996590365, 'sorry': 172.95245998452944, 'bears': 54839.581382576354, 'happens': 129.04012260744702, 'every': 1622.260915177

In [ ]:
class data:

    def __init__(self):
        self.sentence = []
        self.neg = 0
        self.neu = 0
        self.pos = 0
        self.compound = 0
        self.bullish = 0
        self.bearish = 0
        self.likes = 0
    def add_sentence(self, sentence, sentiment, likes):
        self.sentence.append(sentence)

        if sentiment == 'Bullish':
            self.bullish += 1
        elif sentiment == 'Bearish':
            self.bearish += 1
        self.likes += likes

In [ ]:
def sentiment_score(text):
    sentiment_scores = analyzer.polarity_scores(text)
    return sentiment_scores['compound']

In [ ]:
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'http\S+|www\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove @ mentions and # hashtags
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)  # Remove special characters
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

def tokenize(text):
    return word_tokenize(text)

def remove_specific_words(tokens, words_to_remove):
    return [word for word in tokens if word not in words_to_remove]
def concatenate_dates():

    file_path = '/TSLA_big.csv'
    df = pd.read_csv(file_path)



    dated_sentence = dict()
    dates = df['created_at']
    body = df['body']
    likes = df['likes']
    entities = df['entities']


    for i in range(len(dates)):


        preprocessed_data = clean_text(body[i])
        preprocessed_data = remove_stopwords(preprocessed_data)
        preprocessed_data = tokenize(preprocessed_data)
        preprocessed_data = [(remove_specific_words(text, ['tsla'])) for text in preprocessed_data]

        if dates[i] in dated_sentence:
            if entities[i] == 'Bullish' and likes[i] != None:
                dated_sentence[dates[i]].add_sentence(preprocessed_data, entities[i], likes[i])
            if entities[i] == 'Bearish' and likes[i] != None:
                dated_sentence[dates[i]].add_sentence(preprocessed_data, entities[i], likes[i])
            if entities[i] == 'Bullish' and likes[i] == None:
                dated_sentence[dates[i]].add_sentence(preprocessed_data, entities[i], 0)
            if entities[i] == 'Bearish' and likes[i] == None:
                dated_sentence[dates[i]].add_sentence(preprocessed_data, entities[i], 0)
        else:
            dated_sentence[dates[i]] = data()
            if entities[i] == 'Bullish' and likes[i] != None:
                dated_sentence[dates[i]].add_sentence(preprocessed_data, entities[i], likes[i])
            if entities[i] == 'Bearish' and likes[i] != None:
                dated_sentence[dates[i]].add_sentence(preprocessed_data, entities[i], likes[i])
            if entities[i] == 'Bullish' and likes[i] == None:
                dated_sentence[dates[i]].add_sentence(preprocessed_data, entities[i], 0)
            if entities[i] == 'Bearish' and likes[i] == None:
                dated_sentence[dates[i]].add_sentence(preprocessed_data, entities[i], 0)
    return dated_sentence

# dates_df = concatenate_dates()


In [ ]:
def make_dated_csv(df1):
    print("started")
    filename = './TSLA_2020_2022_dated2.csv'
    dated_sentence = df1
    df = pd.DataFrame(columns=['date', 'bullish', 'bearish', 'neg' , 'neu', 'pos', 'compound'])
    print("data frame created")
    import time
    start_time = time.time()

    dates = []
    bullish = []
    bearish = []

    neu = []
    pos = []
    neg = []
    compound = []
    Sentiment_score = []
    number_of_iterations =  0
    for date in dated_sentence:
        print(date)
        dates.append(date)
        bullish.append(dated_sentence[date].bullish)
        bearish.append(dated_sentence[date].bearish)
        words = []
        neuT = 0
        posT = 0
        negT = 0
        compoundT = 0
        tt = 0
        #print(len(dated_sentence[date].sentence))
        for sen in dated_sentence[date].sentence:

            for wordT in sen:
                words.append(''.join(wordT))
            scored = sentiment_score(' '.join(words))
            neuT += scored['neu']
            posT += scored['pos']
            negT += scored['neg']
            compoundT += scored['compound']

        neu.append(neuT)
        pos.append(posT)
        neg.append(negT)
        compound.append(compoundT)


    df['date'] = dates
    df['bullish'] = bullish
    df['bearish'] = bearish
    df['neu'] = neu
    df ['pos'] =pos
    df ['neg'] = neg
    df ['compound'] = compound

    df.to_csv(filename,header=True, index=False)
    print( " time taken = ", time.time() - start_time)

# make_dated_csv(dates_df)


In [ ]:
print(sentiment_score("$SPY If $AAPL and $TSLA are down tomorrow good luck going higher"))



1.0


In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Ensure you have the necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Define text cleaning functions
def clean_text(text):
    if not isinstance(text, str):
        return ''
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'http\S+|www\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove @ mentions and # hashtags
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)  # Remove special characters
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

def tokenize(text):
    return word_tokenize(text)

def remove_specific_words(tokens, words_to_remove):
    return [word for word in tokens if word not in words_to_remove]

# Load the CSV data with error handling
file_path = '/TSLA_big.csv'  # Replace with your actual file path if different
data = pd.read_csv(file_path, on_bad_lines='skip', engine='python')

# Remove NaN values and ensure all entries in 'body' are strings
data['body'] = data['body'].fillna('').astype(str)

# Apply text cleaning functions
data['body'] = data['body'].apply(clean_text)
data['body'] = data['body'].apply(remove_stopwords)
data['cleaned_tokens'] = data['body'].apply(tokenize)

# You can add specific words to remove here
words_to_remove = []  # Add any specific words you want to remove
data['cleaned_tokens'] = data['cleaned_tokens'].apply(lambda x: remove_specific_words(x, words_to_remove))

# Convert tokens back to string for saving
data['body'] = data['cleaned_tokens'].apply(lambda x: ' '.join(x))

# Drop the temporary columns used for processing
data.drop(columns=['cleaned_tokens'], inplace=True)

# Save the cleaned data to a new CSV file
cleaned_file_path = '/TSLA_big_cleaned.csv'  # Path to save the cleaned data
data.to_csv(cleaned_file_path, index=False)

print(f'Cleaned data saved to {cleaned_file_path}')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


FileNotFoundError: [Errno 2] No such file or directory: '/TSLA_big.csv'

In [ ]:


# Load the CSV data
file_path = '/content/TSLA_big_cleaned.csv'  # Replace with your actual file path if different
data = pd.read_csv(file_path)

# Ensure all entries in 'body' are strings and handle missing values
data['body'] = data['body'].fillna('').astype(str)
# Filter out rows where 'entities' is NA
data = data.dropna(subset=['entities'])

# Function to label sentiment based on compound score
def label_sentiment(compound_score):
    return 'Bullish' if compound_score >= 0 else 'Bearish'

# Apply sentiment analysis to each row
data['compound_score'] = data['body'].apply(sentiment_score)
data['predicted_entities'] = data['compound_score'].apply(label_sentiment)

# Calculate accuracy
correct_predictions = (data['entities'] == data['predicted_entities']).sum()
accuracy = correct_predictions / len(data)

# Find mismatched sentiments
mismatched_df = data[data['entities'] != data['predicted_entities']]

# Save the mismatched comments to a CSV file
mismatched_output_file_path = '/mismatched_comments.csv'  # Path to save the mismatched comments
mismatched_df.to_csv(mismatched_output_file_path, index=False, columns=['body', 'entities', 'predicted_entities', 'compound_score'])


# Save the results to a new CSV file
output_file_path = '/SA_Performance_Big_exp1.csv'  # Path to save the results
data.to_csv(output_file_path, index=False, columns=['body', 'entities', 'predicted_entities', 'compound_score'])

bearish_comments = data[data['entities'] == 'Bearish']
correct_bearish_predictions = (bearish_comments['entities'] == bearish_comments['predicted_entities']).sum()
bearish_accuracy = correct_bearish_predictions / len(bearish_comments)

# Print the accuracy

print(f'Bearish Accuracy: {bearish_accuracy * 100:.2f}%')

# Print the accuracy
print(f'Overall Accuracy: {accuracy * 100:.2f}%')
print(f'Results saved to {output_file_path}')

Bearish Accuracy: 22.09%
Overall Accuracy: 78.15%
Results saved to /SA_Performance_Big_exp1.csv


In [ ]:
import pandas as pd                           ####  CODE TO GIVE DICTIONARY NEGETIVE WEIGHTS!!!

# Load the CSV file
file_path = '/Tesla_dectionary_1_umang.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(file_path)

# Multiply negative Sentiment_Score by 100
df['Sentiment_Score'] = df['Sentiment_Score'].apply(lambda x: x * 10000 if x < 0 else x)

# Save the modified dataframe to a new CSV file
output_file_path = '/content/neg_10000.csv'  # Replace with the desired output path
df.to_csv(output_file_path, index=False)

print("Modified CSV file saved successfully.")

Modified CSV file saved successfully.


In [ ]:
### CODE WITH AN LLM...